In [1]:
# Import packages

import TS_Custom
from TS_Custom import *

        
# name of csv file 
backtestingresults_filename = "zlogs_alphaedge_backtestingresults_FULL" + ".csv"
ae_traderesults_filename = "zlogs_alphaedge_backtestingrestuls_ONLY_TRADES" + ".csv"


# Print header row output to csv file 
#printOrderHeaderRowtoCSV(orderlog_filename)

# Global variables
d = {'a': [], 'b': []}

minutes = 5     
barsBack = 2600
#quantity = int(3800/(df['Close'][ind]))
quantity = 5
#barsBack=int(barssince7am(minutes))

def getBacktestingResult(symbol):
    #for symbol in symbol_list :
    try:

        # get price history from tradestation
        
        df = getPriceHistory_TS(symbol,str(barsBack), str(minutes))
        
        #convert current candles to heiken-ashi
        df = getHA(df)
            
        ema9 = df['ema9']
        ema21 = df['ema21']
        ema50 = df['ema50']
        ema200 = df['ema200']  
        
        atr = df['ATR']
        rsi = df['RSI']
        stddev = df['stddeviation']
        
        
        vwap = df['vwap']
        upper =  df['upper_bb']
        middle = df['middle_bb']
        lower = df['lower_bb']
        
        df = Supertrend(df, 10, 1)
        df = UTBot_Sell(df,2,1,False)
        df = UTBot_Buy(df,2,300,False)
        
                   
        
        df['Position'] = 0
        df['OrderType'] = "" 
        tradetype = ""
        avgprice = ""
        
        
        for ind in df.index:
            if (ind>0):

                dt=datetime.strptime(pd.to_datetime(df['TimeStamp'][ind]).strftime('%m-%d-%Y %H:%M:%S'),\
                                                 '%m-%d-%Y %H:%M:%S')
                #Buy Condition
                                    
                if (
                    #(df['ema9_angle'][ind] >= 15) and \
                    #(df['ema9_angle'][ind] > df['ema9_angle'][ind-1]) and \
                    #(df['linear_reg_slope'][ind] > df['linear_reg_slope'][ind-1]) and \
                    #(df['linear_reg_slope'][ind] > 0) and \
                    #(df['K_slow'][ind] > df['D_slow'][ind]) and \
                    #(df['K_slow'][ind] > df['K_slow'][ind-1])and \
                    #(df['K_slow'][ind] < 85) and \
                    #(rsi[ind] <= 80) and \
                    (df['utbot_signal_BUY'][ind] == "BUY") and \
                    (df['ST_Lowerband'][ind]> 0) and \
                    (tradetype=="")):
                    
                    
                    df['Position'][ind] = -quantity
                    df['OrderType'][ind] = "BUYSTOCK"
                    tradetype = "BUYSTOCK"
                    avgprice = df['Close'][ind]
                    triggerprice = round(quantity * 0.50 * stddev[ind],2)
                    continue
                
                #Sell Condition
                if (tradetype=="BUYSTOCK"):
                   
                    openpl = float(pd.to_numeric(df['HA_Close'][ind]) - pd.to_numeric(avgprice))
                    openpl_percent = float(((pd.to_numeric(df['HA_Close'][ind]) - pd.to_numeric(avgprice))/pd.to_numeric(avgprice))*100)
                    #remove last open trade before results or before eod
                    dt=datetime.strptime(pd.to_datetime(df['TimeStamp'][ind]).strftime('%m-%d-%Y %H:%M:%S'),\
                                                 '%m-%d-%Y %H:%M:%S')
                    
                    #print ('openpl_percent for buy is ',openpl_percent)
                    #if (openpl >= 0.003 * pd.to_numeric(avgprice)) or (openpl_percent <= -0.1) :
                        #or (dt.hour==14 and dt.minute==55)) :
                    
                    #if ((df['linear_reg_slope'][ind-2] < df['linear_reg_slope'][ind - 1]) and \
                        #(df['linear_reg_slope'][ind-1] < df['linear_reg_slope'][ind])) :
                    
                    #if (openpl_percent >= 1.00) or (openpl_percent <= -0.50) :
                    if (openpl>=triggerprice or df['ST_Upperband'][ind] > 0) :
                    #if (openpl_percent >= 0.1) or (openpl_percent <= -0.50):
                    #if (df['utbot_signal_SELL'][ind] == "SELL") :
                        
                        df['Position'][ind] = quantity
                        df['OrderType'][ind] = "SELLSTOCK"
                        tradetype = "" 
                        avgprice = ""     
                        continue
    
                    
                #SellShort Condition
                
                if (
                    #(df['ema9_angle'][ind] <= -5) and \
                    #(df['ema9_angle'][ind] < df['ema9_angle'][ind-1]) and \
                    #(df['linear_reg_slope'][ind] < df['linear_reg_slope'][ind-1]) and \
                    #(df['linear_reg_slope'][ind] < 0) and \
                    #(df['HA_Close'][ind] < df['HA_Open'][ind]) and \
                    #(df['K_slow'][ind] < df['D_slow'][ind]) and \
                    #(df['K_slow'][ind] < df['K_slow'][ind-1]) and \
                    #(df['K_slow'][ind] > 15) and \
                    #(rsi[ind] >= 25) and \
                    (df['utbot_signal_SELL'][ind] == "SELL") and \
                    (df['ST_Upperband'][ind]> 0) and \
                    (tradetype=="")): 
              
                    df['Position'][ind] = quantity
                    df['OrderType'][ind] = "SELLSHORTSTOCK"
                    tradetype = "SELLSHORTSTOCK"
                    avgprice = df['Close'][ind]
                    triggerprice = round(quantity * 0.50 * stddev[ind],2)
                    continue
                    
                
                #BuyToCover Condition
                
                
                if (tradetype=="SELLSHORTSTOCK"):
                    
                    openpl = float(pd.to_numeric(avgprice) - pd.to_numeric(df['HA_Close'][ind]))
                    openpl_percent = float(((pd.to_numeric(avgprice) - pd.to_numeric(df['HA_Close'][ind]))/pd.to_numeric(avgprice))*100)
                    
                    #print ('openpl_percent for sell is ',openpl_percent)
                    #remove last open trade before results or before eod
                    dt=datetime.strptime(pd.to_datetime(df['TimeStamp'][ind]).strftime('%m-%d-%Y %H:%M:%S'),\
                                                 '%m-%d-%Y %H:%M:%S')
                    
                    #if (openpl >= 0.0030 * pd.to_numeric(avgprice)) or (openpl_percent <= -0.1) :
                        #(dt.hour==14 and dt.minute==55)) :                    
                    
                    #if ((df['linear_reg_slope'][ind-2] > df['linear_reg_slope'][ind - 1]) and \
                        #(df['linear_reg_slope'][ind-1] > df['linear_reg_slope'][ind])) :
                    
                    #if (openpl_percent >= 1.00) or (openpl_percent <= -0.50) :
                    if (openpl>=triggerprice or df['ST_Lowerband'][ind] > 0) :  
                    #if (openpl_percent >= 0.1) or (openpl_percent <= -0.50):
                    #if (df['utbot_signal_BUY'][ind] == "BUY") :
                        
                        df['Position'][ind] = -quantity
                        df['OrderType'][ind] = "BUYTOCOVERSTOCK"
                        tradetype = "" 
                        avgprice = ""
                        continue
    

        df['Value']=df['Position']*df['Close']
        df['Symbol'] = symbol
        df = df[['Symbol','TimeStamp','Close','Open','High','Low','ema9','ema21','ema50','ema200','vwap',
                 'ATR','K_slow','D_slow','ST_Lowerband','ST_Upperband','utbot_signal_BUY','utbot_signal_SELL','Position','Value','OrderType']]
        #pd.set_option('display.max_rows', None)
        #print(df)
        
        
        
        df2 = df.loc[((df['Position'] == -quantity ) | (df['Position'] == quantity))]
        df2['P/L'] = 0.00
        df2.reset_index(inplace=True)
        for ind in df2.index:
            if (ind>0):
                if ((df2['OrderType'][ind] == "SELLSTOCK") or (df2['OrderType'][ind] == "BUYTOCOVERSTOCK")):
                    df2['P/L'][ind] = df2['Value'][ind] + df2['Value'][ind-1]
                    #ind = ind + 2
            
        profit_loss_symbol = df2['P/L'].sum()        
        print('P/L =',profit_loss_symbol,'for',symbol)
        
        
        d['a'].append(profit_loss_symbol) 
        d['b'].append(df['Close'][0]*quantity)

                
        df.to_csv(backtestingresults_filename, mode = 'a', index=False)
        df2.to_csv(ae_traderesults_filename, mode = 'a', index=False)
        
        time.sleep(1)      
        

    except Exception as e:
        traceback.print_exc()
        print(e)
        
     
        
        

#Backtest Strategy

symbol_list = pd.read_csv('tickers_backtesting.csv', header=None)[0].tolist()
print(symbol_list)



# Time starts from here
start = time.time()

import concurrent.futures

with concurrent.futures.ThreadPoolExecutor(max_workers=len(symbol_list)) as executor:
    #print(threading.currentThread().ident)
    executor.map(getBacktestingResult,symbol_list)

# Total time calculated
print('The program takes ', time.time()-start, 'seconds.')


 
total_profit_loss = sumOfList(d['a'], len(d['a']))
total_cost = sumOfList(d['b'], len(d['b']))

print("---------------------------------------------")
print("Number of Days: ", (barsBack/96))
print("Number of Shares per Stock: ", quantity)
print("Total Profit and Loss: $", round(total_profit_loss,2))
print("Total Cost: $", round(total_cost,2))
print("Total %P/L: ", round(((total_profit_loss/total_cost) * 100),2),'%')

print("---------------------------------------------")
        
print("**************************************************************")
   




['MU', 'ZM']
Last Date was  06-13-2022
Date was MISMATCH. Had to reset BARSBACK for  ZM
Date was MISMATCH. DATAFRAME is EMPTY ZM
P/L = -51.500000000000114 for ZM
Last Date was  06-22-2022
Date was MISMATCH. Had to reset BARSBACK for  MU
Date was MISMATCH. DATAFRAME is EMPTY MU
P/L = -276.7999999999997 for MU
The program takes  22.395391702651978 seconds.
---------------------------------------------
Number of Days:  27.083333333333332
Number of Shares per Stock:  5
Total Profit and Loss: $ -328.3
Total Cost: $ 812.15
Total %P/L:  -40.42 %
---------------------------------------------
**************************************************************
